In [ ]:
!pip install ucimlrepo
!pip install pyMetaheuristic
!pip install torchmetrics

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchmetrics import Accuracy

from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from scipy import optimize
import numpy as np
import pandas as pd
import math
import random

#Feature Selection

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# fetch dataset
isolet = fetch_ucirepo(id=54)

# data (as pandas dataframes)
X = isolet.data.features
y = isolet.data.targets

y -=1

In [ ]:
class AverageMeter():
    """Computes and stores the average and current loss value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train_one_epoch(model, train_loader, loss_fn, optimizer, epoch=None):
  model.train()
  loss_train = AverageMeter()
  acc_train = Accuracy(task='multiclass', num_classes=26).to(device)
  #with tqdm(train_loader, unit='batch') as tepoch:
  for inputs, targets in train_loader:
      #if epoch is not None:
      # tepoch.set_description(f'Epoch {epoch}')
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)
      loss = loss_fn(outputs, targets)

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      loss_train.update(loss.item())
      acc_train(outputs, targets.int())

      #tepoch.set_postfix(loss=loss_train.avg,
      #                   accuracy=100.*acc_train.compute().item())
  return model, loss_train.avg, acc_train.compute().item()

In [ ]:
def evaluate(model, valid_loader, loss_fn):
  model.eval()
  with torch.no_grad():
    loss_valid = AverageMeter()
    acc_valid = Accuracy(task='multiclass', num_classes=26).to(device)
    for i, (inputs, targets) in enumerate(valid_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)
      loss = loss_fn(outputs, targets)

      loss_valid.update(loss.item())
      acc_valid(outputs, targets.int())

  return loss_valid.avg, acc_valid.compute()

In [ ]:
class RecurrentModel(nn.Module):
  def __init__(self, model, input_size, hidden_size, num_layers, bidirectional):
    super().__init__()
    self.rnn = model(input_size = input_size,
                      hidden_size = hidden_size,
                      num_layers = num_layers,
                      bidirectional = bidirectional,
                      batch_first = True)
    self.fc = nn.LazyLinear(26)

  def forward(self, x):
    outputs, _ = self.rnn(x)
    #print(outputs.shape)
    y = self.fc(outputs) # out: many[:, -1, :]
    #y = y.mean(dim=1)
    return y

In [ ]:
def si_opt(method = 'aro', obj = lambda x : x**2 , lb =[- np.inf] , ub = [np.inf], dim = 1, npop = 50, max_it = 100):
    '''
    4 Swarm INtelligence Optimizer: aro, woa, gwo, pso
    '''
    if method == 'aro' :
        from torch import randperm

        def space_bound(X, Up, Low):
            dim = len(X)
            S = (X > Up) + (X < Low)
            res = (np.random.rand(dim) * (np.array(Up) - np.array(Low)) + np.array(Low)) * S + X * (~S)
            return res


        if len(lb) == 1:
            lb = lb * dim
            ub = ub * dim

        pop_pos = np.zeros((npop, dim))
        for i in range(dim):
            pop_pos[:, i] = np.random.rand(npop) * (ub[i] - lb[i]) + lb[i]
        pop_fit = np.zeros(npop)
        for i in range(npop):
            pop_fit[i] = obj(pop_pos[i, :])

        best_f = float('inf')
        best_x = []
        for i in range(npop):
            if pop_fit[i] <= best_f:
                best_f = pop_fit[i]
                best_x = pop_pos[i, :]

        for it in range(max_it):
            direct1 = np.zeros((npop, dim))
            direct2 = np.zeros((npop, dim))
            theta = 2 * (1 - (it+1) / max_it)
            for i in range(npop):
                L = (np.e - np.exp((((it+1) - 1) / max_it) ** 2)) * (np.sin(2 * np.pi * np.random.rand())) # Eq.(3)
                rd = np.floor(np.random.rand() * (dim))
                rand_dim = randperm(dim)
                direct1[i, rand_dim[:int(rd)]] = 1
                c = direct1[i,:]  #Eq.(4)
                R = L * c # Eq.(2)
                A = 2 * np.log(1 / np.random.rand()) * theta #Eq.(15)

                if A>1:
                  K=np.r_[0:i,i+1:npop]
                  RandInd=(K[np.random.randint(0,npop-1)])
                  newPopPos = pop_pos[RandInd, :] + R * (pop_pos[i, :] - pop_pos[RandInd, :])+round(0.5 * (0.05 +np.random.rand())) * np.random.randn()
                  # Eq.(1)
                else:
                    ttt = int(np.floor(np.random.rand() * dim))
                    direct2[i, ttt] = 1
                    gr = direct2[i,:] #Eq.(12)
                    H = ((max_it - (it+1) + 1) / max_it) * np.random.randn() # % Eq.(8)
                    b = pop_pos[i,:] + H * gr * pop_pos[i,:] # % Eq.(13)
                    newPopPos = pop_pos[i,:]+ R* (np.random.rand() * b - pop_pos[i,:]) #Eq.(11)

                newPopPos = space_bound(newPopPos, ub, lb)
                newPopFit = obj(newPopPos)

                if newPopFit < pop_fit[i]:
                    pop_fit[i] = newPopFit
                    pop_pos[i, :] = newPopPos

                if pop_fit[i] < best_f:
                    best_f = pop_fit[i]
                    best_x = pop_pos[i, :]

            print(f'The Minimum value at iteration {it} is equal to: {best_f}')

        print()

        return best_x, best_f

    elif method == "woa" :
        import random
        import copy
        import sys
        import math

        class whale:
            def __init__(self, fitness, dim, minx, maxx, seed):
                self.rnd = random.Random(seed)
                self.position = [0.0 for i in range(dim)]

                for i in range(dim):
                    self.position[i] = ((maxx - minx) * self.rnd.random() + minx)

                self.fitness = fitness(self.position)


        def woa(fitness, max_iter, n, dim, minx, maxx):
            rnd = random.Random(0)

            # create n random whales
            whalePopulation = [whale(fitness, dim, minx, maxx, i) for i in range(n)]

            # compute the value of best_position and best_fitness in the whale Population
            Xbest = [0.0 for i in range(dim)]
            Fbest = sys.float_info.max

            for i in range(n): # check each whale
                if whalePopulation[i].fitness < Fbest:
                    Fbest = whalePopulation[i].fitness
                    Xbest = copy.copy(whalePopulation[i].position)

            # main loop of woa
            Iter = 0
            while Iter < max_iter:
                # linearly decreased from 2 to 0
                a = 2 * (1 - Iter / max_iter)
                a2 = -1+Iter*((-1)/max_iter)

                for i in range(n):
                    A = 2 * a * rnd.random() - a
                    C = 2 * rnd.random()
                    b = 1
                    l = (a2-1)*rnd.random()+1;
                    p = rnd.random()

                    D = [0.0 for i in range(dim)]
                    D1 = [0.0 for i in range(dim)]
                    Xnew = [0.0 for i in range(dim)]
                    Xrand = [0.0 for i in range(dim)]

                    if p < 0.5:
                        if abs(A) > 1:
                            for j in range(dim):
                                D[j] = abs(C * Xbest[j] - whalePopulation[i].position[j])
                                Xnew[j] = Xbest[j] - A * D[j]
                        else:
                            p = random.randint(0, n - 1)
                            while (p == i):
                                p = random.randint(0, n - 1)

                            Xrand = whalePopulation[p].position

                            for j in range(dim):
                                D[j] = abs(C * Xrand[j] - whalePopulation[i].position[j])
                                Xnew[j] = Xrand[j] - A * D[j]
                    else:
                        for j in range(dim):
                            D1[j] = abs(Xbest[j] - whalePopulation[i].position[j])
                            Xnew[j] = D1[j] * math.exp(b * l) * math.cos(2 * math.pi * l) + Xbest[j]

                    for j in range(dim):
                        whalePopulation[i].position[j] = Xnew[j]

                for i in range(n):
                # if Xnew < minx OR Xnew > maxx
                # then clip it
                    for j in range(dim):
                        whalePopulation[i].position[j] = max(whalePopulation[i].position[j], minx)
                        whalePopulation[i].position[j] = min(whalePopulation[i].position[j], maxx)

                    whalePopulation[i].fitness = fitness(whalePopulation[i].position)

                    if (whalePopulation[i].fitness < Fbest):
                        Xbest = copy.copy(whalePopulation[i].position)
                        Fbest = whalePopulation[i].fitness


                print(f'Minimum Value at iteration {Iter} is equal to: {Fbest}')
                Iter += 1
                # end-while

            # returning the best solution
            return Xbest, Fbest

        best_x, best_f = woa(obj,max_it, npop, dim, lb[0], ub[0])
        return np.array(best_x) , best_f


    elif method == 'gwo' :

        import random
        class solution:
            def __init__(self):
                self.best = 0
                self.bestIndividual = []
                self.lb = 0
                self.ub = 0
                self.dim = 0
                self.popnum = 0
                self.maxiers = 0

        def gwo(objf, lb, ub, dim, SearchAgents_no, Max_iter):

            # initialize alpha, beta, and delta_pos
            Alpha_pos = np.zeros(dim)
            Alpha_score = float("inf")

            Beta_pos = np.zeros(dim)
            Beta_score = float("inf")

            Delta_pos = np.zeros(dim)
            Delta_score = float("inf")

            if not isinstance(lb, list):
                lb = [lb] * dim
            if not isinstance(ub, list):
                ub = [ub] * dim

            if len(lb) == 1:
                lb = lb * dim
            if len(ub) == 1:
                ub = ub * dim

            # Initialize the positions of search agents
            Positions = np.zeros((SearchAgents_no, dim))
            for i in range(dim):
                Positions[:, i] = (
                    np.random.uniform(0, 1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i]
                )

            s = solution()
            # Main loop
            for l in range(0, Max_iter):
                a_pos = 0
                for i in range(0, SearchAgents_no):

                    # Return back the search agents that go beyond the boundaries of the search space
                    for j in range(dim):
                        Positions[i, j] = np.clip(Positions[i, j], lb[j], ub[j])

                    # Calculate objective function for each search agent
                    fitness = objf(Positions[i, :])

                    # Update Alpha, Beta, and Delta
                    if fitness < Alpha_score:
                        a_pos = i
                        Delta_score = Beta_score  # Update delte
                        Delta_pos = Beta_pos.copy()
                        Beta_score = Alpha_score  # Update beta
                        Beta_pos = Alpha_pos.copy()
                        Alpha_score = fitness
                        # Update alpha
                        Alpha_pos = Positions[i, :].copy()

                    if fitness > Alpha_score and fitness < Beta_score:
                        Delta_score = Beta_score  # Update delte
                        Delta_pos = Beta_pos.copy()
                        Beta_score = fitness  # Update beta
                        Beta_pos = Positions[i, :].copy()

                    if fitness > Alpha_score and fitness > Beta_score and fitness < Delta_score:
                        Delta_score = fitness  # Update delta
                        Delta_pos = Positions[i, :].copy()

                a = 2 - l * ((2) / Max_iter)
                # a decreases linearly fron 2 to 0

                # Update the Position of search agents including omegas
                for i in range(0, SearchAgents_no):
                    for j in range(0, dim):

                        r1 = random.random()  # r1 is a random number in [0,1]
                        r2 = random.random()  # r2 is a random number in [0,1]

                        A1 = 2 * a * r1 - a
                        # Equation (3.3)
                        C1 = 2 * r2
                        # Equation (3.4)

                        D_alpha = abs(C1 * Alpha_pos[j] - Positions[i, j])
                        # Equation (3.5)-part 1
                        X1 = Alpha_pos[j] - A1 * D_alpha
                        # Equation (3.6)-part 1

                        r1 = random.random()
                        r2 = random.random()

                        A2 = 2 * a * r1 - a
                        # Equation (3.3)
                        C2 = 2 * r2
                        # Equation (3.4)

                        D_beta = abs(C2 * Beta_pos[j] - Positions[i, j])
                        # Equation (3.5)-part 2
                        X2 = Beta_pos[j] - A2 * D_beta
                        # Equation (3.6)-part 2

                        r1 = random.random()
                        r2 = random.random()

                        A3 = 2 * a * r1 - a
                        # Equation (3.3)
                        C3 = 2 * r2
                        # Equation (3.4)

                        D_delta = abs(C3 * Delta_pos[j] - Positions[i, j])
                        # Equation (3.5)-part 3
                        X3 = Delta_pos[j] - A3 * D_delta
                        # Equation (3.5)-part 3

                        Positions[i, j] = (X1 + X2 + X3) / 3  # Equation (3.7)

                print(["At iteration " + str(l) + " the best fitness is " + str(Alpha_score)])

            s.bestIndividual = Alpha_pos
            s.best = Alpha_score
            print()
            return s

        solve = gwo(obj, lb, ub, dim, npop, max_it)
        return solve.bestIndividual , solve.best

    elif method == 'pso' :

        from pyMetaheuristic.algorithm import particle_swarm_optimization as pso

        if len(lb) == 1:
            lb = lb * dim
        if len(ub) == 1:
            ub = ub * dim

        parameters = {
        'swarm_size': npop,
        'min_values': lb,
        'max_values': ub,
        'iterations': max_it - 1,
        'decay': 0,
        'w': 0.9,
        'c1': 2,
        'c2': 2
        }

        solve = pso(target_function = obj, **parameters)
        x_best = solve[:-1]
        f_best = solve[-1]
        return x_best, f_best

    else:
      raise Exception("You should choose one of 4 suggested methods")

In [ ]:
def binary_conversion(x, thresh = 0.5):
    for i in range(len(x)):
        if x[i] > thresh:
            x[i] = 1
        else:
            x[i] = 0
    return x

def feature_selector(binary_vec):
    features = []
    for i in range(len(binary_vec)):
        if binary_vec[i] == 1:
            features.append(i)
    return features


def fs(v):
    ''' feature selector '''
    binary_vector = binary_conversion(v)
    x = X.iloc[:, feature_selector(binary_vector)]

    x_train, x_valid, y_train, y_valid = train_test_split(x, y, train_size=0.8, random_state=24)
    x_train = torch.FloatTensor(x_train.values)
    y_train = torch.LongTensor(y_train['class'].values)

    x_valid = torch.FloatTensor(x_valid.values)
    y_valid = torch.LongTensor(y_valid['class'].values)

    mu = x_train.mean(dim=0)
    std = x_train.std(dim=0)

    x_train = (x_train - mu) / std
    x_valid = (x_valid - mu) / std

    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, 20, True)

    valid_dataset = TensorDataset(x_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, 40, False)


    num_feats = x.shape[1]
    model = RecurrentModel(nn.LSTM, num_feats, 100, 1, False)

    optimizer = optim.Adam(model.parameters(), lr = 0.001)
    loss_fn = nn.CrossEntropyLoss()
    best_loss = torch.inf
    num_epochs = 5

    for epoch in range(num_epochs):
  # Train
        model, loss_train, acc_train = train_one_epoch(model,
                                                 train_loader,
                                                 loss_fn,
                                                 optimizer,
                                                 epoch)
  # Validation
        loss_valid, acc_valid = evaluate(model,
                                     valid_loader,
                                     loss_fn)

        if loss_train < best_loss:
            best_loss = loss_train

    return 0.5 * best_loss + 0.5 * (num_feats / (617 - num_feats))

In [ ]:
sol = si_opt('aro', fs, [0], [1], 617, npop = 10, max_it = 30)
binary_v = sol[0]

# Hyperparameter Tuning

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
x = X.iloc[:, feature_selector(binary_v)]

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, train_size=0.8, random_state=42)

x_train = torch.FloatTensor(x_train.values)
y_train = torch.LongTensor(y_train['class'].values)

x_valid = torch.FloatTensor(x_valid.values)
y_valid = torch.LongTensor(y_valid['class'].values)

mu = x_train.mean(dim=0)
std = x_train.std(dim=0)

x_train = (x_train - mu) / std
x_valid = (x_valid - mu) / std

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, 20, True)

valid_dataset = TensorDataset(x_valid, y_valid)
valid_loader = DataLoader(valid_dataset, 20, True)

In [ ]:
class TransformerModel(nn.Module):

  def __init__(self, num_enc, d_model, nhead, nd_feed, dropout = 0.1, af = 'relu', num_cls = 26):
    super().__init__()

    self.encoder = nn.TransformerEncoderLayer(d_model,
                                  nhead,
                                  nd_feed * d_model,
                                  dropout,
                                  af,
                                  batch_first=True,
                                  device = 'cuda')

    self.model = nn.TransformerEncoder(self.encoder, num_enc)

    # classifier
    self.fc = nn.LazyLinear(num_cls)

    # input layer
    self.linear = nn.LazyLinear(d_model)
    self.bn = nn.LazyBatchNorm1d()

  def forward(self, x):
    x = self.bn(self.linear(x)).relu()
    y = self.model(x)
    y = self.fc(y)
    return y

In [ ]:
model = TransformerModel(1, 128, 8, 4, 0.1, 'relu').to(device)
loss_fn = nn.CrossEntropyLoss()

lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr = lr)

best_valid_loss = torch.inf

In [ ]:
losses = []
for _ in range(10):
    model = TransformerModel(1, 128, 8, 4, 0.1, 'relu').to(device)
    loss_fn = nn.CrossEntropyLoss()
    lr = 1e-3
    optimizer = optim.Adam(model.parameters(), lr = lr)
    best_valid_loss = torch.inf
    num_epochs = 10
    for epoch in range(num_epochs):
        # Train
        model, loss_train, acc_train = train_one_epoch(model,
                                          train_loader,
                                          loss_fn,
                                          optimizer,
                                          epoch)

        # Validation
        loss_valid, acc_valid = evaluate(model,
                                      valid_loader,
                                      loss_fn)

        if loss_valid < best_valid_loss:
            best_valid_loss = loss_valid
    print(f'Best Valid Loss at iteration {_} = {best_valid_loss:.4}')
    print()
    losses.append(best_valid_loss)

In [ ]:
print(f'Average of best valid loss = {np.mean(losses)}')
print(f'Std of best valid loss = {np.std(losses)}')

In [ ]:
def hp_tuning(p):
    num_enc = int(6 * p[0]) + 1
    nhead = 2 ** (int(6 * p[1]) + 1)
    nd_feed = int(4 * p[2]) + 1
    dropout = p[3]
    af = 'relu' if p[4] >= 0.5 else 'gelu'
    lr = p[5] / 100

    model = TransformerModel(num_enc, 128, nhead, nd_feed, dropout, af).to(device)
    optimizer = optim.Adam(model.parameters(), lr = lr)
    best_loss = torch.inf

    num_epochs = 10

    for epoch in range(num_epochs):
        # Train
        model, loss_train, acc_train = train_one_epoch(model,
                                             train_loader,
                                             loss_fn,
                                             optimizer)

        # Validation
        loss_valid, acc_valid = evaluate(model,
                                     valid_loader,
                                     loss_fn)

        if loss_valid < best_loss:
            best_loss = loss_valid

    return best_loss

In [ ]:
history = []
for _ in range(10):
    if _ == 0:
        print(f'{_ + 1}st Loop:')
    elif _ == 1:
        print(f'{_ + 1}nd Loop:')
    elif _ == 2:
        print(f'{_ + 1}rd Loop:')
    else:
        print(f'{_ + 1}th Loop:')
    ans = si_opt('aro', hp_tuning, [0], [1], 6, npop = 10, max_it = 20)
    err = ans[1]
    enc = int(6 * ans[0][0]) + 1
    h = 2 ** (int(6 * ans[0][1]) + 1)
    feed = (int(4 * ans[0][2]) + 1) * 128
    d = ans[0][3]
    f = 1 if ans[0][4] >= 0.5 else 0
    l = ans[0][-1] / 100
    history.append((enc, h, feed, d, f, l, err))

In [ ]:
print('Results for ARO:')
print()
print(f'Average of Validation Loss = {np.mean([res[-1] for res in history])}')
print(f'Std of Validation Loss = {np.std([res[-1] for res in history])}')
print()
print(f'Average of layers = {np.mean([res[0] for res in history])}')
print(f'Average of attention heads = {np.mean([res[1] for res in history])}')
print(f'Average of feed-forward dimension = {np.mean([res[2] for res in history])}')
print(f'Average of dropout = {np.mean([res[3] for res in history])}')
print(f'prefered activation function = {np.mean([res[4] for res in history])}')
print(f'Average of learning rate = {np.mean([res[5] for res in history])}')